<a href="https://colab.research.google.com/github/kmurdzek/Stock-Price-Prediction-using-EOD-Data-and-TensorFlow/blob/main/trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests
import pandas as pd
import numpy as np
from io import StringIO
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
import os
API_KEY = os.environ["EOD_API_TOKEN"]

In [3]:
def get_eod_data(symbol="AAPL.US", session=None):
    api_token=API_KEY
    if session is None:
        session = requests.Session()
    url = 'https://eodhistoricaldata.com/api/eod/%s' % symbol
    params = {
        "api_token": api_token,  # Replace with your API token
        "from": "2020-01-01",  # Start date for the data
        "to": "2022-12-31",  # End date for the data
        "sort_order": "asc"  # Sort the data in ascending order
    }
    r = session.get(url, params=params)
    if r.status_code == requests.codes.ok:
        df = pd.read_csv(StringIO(r.text), skipfooter=0, parse_dates=[
                         0], index_col=0, engine='python')
        df = transform_dataframe(df)
        return df

    else:
        raise Exception(r.status_code, r.reason, url)


In [5]:
def transform_dataframe(df):
    #add bolinger bands
    df = add_bolinger_bands(df)
    df = calculate_rsi(df)
    df.pop('Adjusted_close')
    df = df.iloc[20:]
    target = df.pop('Close')
    
    numeric_feature_names = ['Open', 'High', 'Low',  'Volume', 'upper_bands', 'lower_bands', 'rsi']
    numeric_features = df[numeric_feature_names]
    # Split the dataset into a training set and a testing set
    x_train, x_test, y_train, y_test = train_test_split(numeric_features, target, test_size=0.2)
    x_train = tf.convert_to_tensor(x_train)
    x_test = tf.convert_to_tensor(x_test)
    normalizer = tf.keras.layers.Normalization(axis=-1)
    normalizer.adapt(x_train)
    normalizer.adapt(x_test)
    model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(units=64, activation="relu"),
    tf.keras.layers.Dense(units=1)])

    model.compile(optimizer='adam',
            loss="mse",metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=1000)
    print("Generate a prediction")
    # Evaluate the model on the testing data
    metrics = model.evaluate(x_test, y_test)
    print(metrics)
    return df

In [6]:
def calculate_rsi(df):
    # Set the number of periods to calculate RSI
    window_length = 14

    # Calculate the differences in closing prices
    df['diff'] = df['Close'].diff().round(2)

    # Calculate Avg. Gains/Losses
    df['gain'] = df['diff'].clip(lower=0).round(2)
    df['loss'] = df['diff'].clip(upper=0).abs().round(2)

    # Get initial Averages
    df['avg_gain'] = df['gain'].rolling(window=window_length, min_periods=window_length).mean()[:window_length+1]
    df['avg_loss'] = df['loss'].rolling(window=window_length, min_periods=window_length).mean()[:window_length+1]
    # Get WMS averages
    # Average Gains
    for i, row in enumerate(df['avg_gain'].iloc[window_length+1:]):
        df['avg_gain'].iloc[i + window_length + 1] =\
            (df['avg_gain'].iloc[i + window_length] *
            (window_length - 1) +
            df['gain'].iloc[i + window_length + 1])\
            / window_length
    # Average Losses
    for i, row in enumerate(df['avg_loss'].iloc[window_length+1:]):
        df['avg_loss'].iloc[i + window_length + 1] =\
            (df['avg_loss'].iloc[i + window_length] *
            (window_length - 1) +
            df['loss'].iloc[i + window_length + 1])\
            / window_length
    # Calculate RS Values
    df['rs'] = df['avg_gain'] / df['avg_loss']
    # Calculate RSI
    df['rsi'] = 100 - (100 / (1.0 + df['rs']))
    return df

In [7]:
def add_bolinger_bands(df):
      # Calculate the Bollinger bands
  df["upper_bands"] = df["Close"].rolling(
      20).mean() + 2 * df["Close"].rolling(20).std()
  df["lower_bands"] = df["Close"].rolling(
      20).mean() - 2 * df["Close"].rolling(20).std()
  return df

In [ ]:
get_eod_data()